### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
import pandas as pd
import numpy as np

# set up a student and a schools df
schools_path = "Resources/schools_complete.csv"
students_path = "Resources/students_complete.csv"
schools_df = pd.read_csv(schools_path)
students_df = pd.read_csv(students_path)
school_student_combo = students_df.merge(schools_df, how='left', on='school_name')
school_student_combo.set_index('Student ID', inplace=True)
school_student_combo.head()

,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
Student ID,,,,,,,,,,
0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)
* Create a dataframe to hold the above results
* Optional: give the displayed data cleaner formatting

In [3]:
# start with number of students
num_students = students_df.count()['student_name']
num_schools = schools_df.count()['school_name']
budget = schools_df.sum()['budget']
math_column = students_df['math_score']
reading_column = students_df['reading_score']
ave_math = round(math_column.mean(), 2)
ave_reading = round(reading_column.mean(), 2)
num_passing_math = math_column[math_column>69].count()
pct_passing_math = round(num_passing_math / num_students * 100, 2)
num_passing_reading = reading_column[reading_column>69].count()
pct_passing_reading = round(num_passing_reading / num_students * 100, 2)

math_and_reading = students_df[['reading_score', 'math_score']]
passing_all = math_and_reading[
    (math_and_reading['reading_score']>69) 
    & (math_and_reading['math_score']>69)
].shape[0]
pct_passing_all = round(passing_all / num_students * 100, 2)

In [4]:
# Build summary df
formatted_num_students = np.array2string(num_students, formatter={'int_kind': lambda x: '{:,}'.format(x)})
formatted_budget = np.array2string(budget, formatter={'int_kind': lambda x: '${:,.2f}'.format(x)})
district_summary_df = pd.DataFrame({
    'Number of Schools': num_schools,
    'Number of Students': formatted_num_students,
    'Total Budget': formatted_budget,
    'Average Math Score': ave_math,
    'Average Reading Score': ave_reading,
    '% Passing Math': pct_passing_math,
    '% Passing Reading': pct_passing_reading,
    '% Overall Passing': pct_passing_all
}, index=[0])
district_summary_df.head()

,Number of Schools,Number of Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


## School Summary
  * Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.) 
  * Create a dataframe to hold the above results

In [5]:
school_students_df = school_student_combo.groupby('school_name').count()['student_name']
school_students_df.rename('Total Students', inplace=True)
school_budget_df = schools_df.set_index('school_name')['budget']
school_budget_df.rename('Total School Budget', inplace=True)
school_type_df = schools_df.set_index('school_name')['type']
school_type_df.rename('School Type', inplace=True)
school_budget_norm_df = round(school_budget_df / school_students_df, 2)
school_budget_norm_df.rename('Per Student Budget', inplace=True)
school_ave_math_df = round(school_student_combo.groupby('school_name').mean()['math_score'],2)
school_ave_math_df.rename('Average Math Score', inplace=True)
school_ave_reading_df = round(school_student_combo.groupby('school_name').mean()['reading_score'],2)
school_ave_reading_df.rename('Average Reading Score', inplace=True)
school_num_passing_math = school_student_combo[school_student_combo['math_score']>69].groupby('school_name').count()['math_score']
school_pct_passing_math = round(school_num_passing_math / school_students_df * 100, 2)
school_pct_passing_math.rename('% Passing Math', inplace=True)
school_num_passing_reading = school_student_combo[school_student_combo['reading_score']>69].groupby('school_name').count()['reading_score']
school_pct_passing_reading = round(school_num_passing_reading / school_students_df * 100, 2)
school_pct_passing_reading.rename('% Passing Reading', inplace=True)

school_passing_all = school_student_combo[
    (school_student_combo['math_score']>69)
    & (school_student_combo['reading_score']>69)
].groupby('school_name').count()['student_name']
school_pct_passing_all = round(school_passing_all / school_students_df * 100, 2)
school_pct_passing_all.rename('% Overall Passing', inplace=True)

# create summary DataFrame
school_summary_df = pd.DataFrame({
    school_type_df.name: school_type_df,
    school_students_df.name: school_students_df,
    school_budget_df.name: school_budget_df.map('${:,.2f}'.format),
    school_budget_norm_df.name: school_budget_norm_df.map('${:,.2f}'.format),
    school_ave_math_df.name: school_ave_math_df,
    school_ave_reading_df.name: school_ave_reading_df,
    school_pct_passing_math.name: school_pct_passing_math,
    school_pct_passing_reading.name: school_pct_passing_reading,
    school_pct_passing_all.name: school_pct_passing_all
})
school_summary_df.index.name = 'School Name'
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,89.23
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54


## Top Performing Schools, by % Overall Passing
* Sort and display the top five performing schools by % overall passing.

In [6]:
top_schools_df = school_summary_df.sort_values('% Overall Passing', ascending=False)
top_schools_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [7]:
top_schools_df.sort_values('% Overall Passing').head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [17]:
students_ninth = students_df[students_df['grade']=='9th'].groupby('school_name').mean()[['math_score', 'reading_score']]
students_tenth = students_df[students_df['grade']=='10th'].groupby('school_name').mean()[['math_score', 'reading_score']]
students_eleventh = students_df[students_df['grade']=='11th'].groupby('school_name').mean()[['math_score', 'reading_score']]
students_twelfth = students_df[students_df['grade']=='12th'].groupby('school_name').mean()[['math_score', 'reading_score']]

# create summary DataFrame
math_scores_by_school = pd.DataFrame({
    '9th': round(students_ninth['math_score'], 1),
    '10th': round(students_tenth['math_score'], 1),
    '11th': round(students_eleventh['math_score'], 1),
    '12th': round(students_twelfth['math_score'], 1)
})
math_scores_by_school

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [18]:
reading_scores_by_school = pd.DataFrame({
    '9th': round(students_ninth['reading_score'], 1),
    '10th': round(students_tenth['reading_score'], 1),
    '11th': round(students_eleventh['reading_score'], 1),
    '12th': round(students_twelfth['reading_score'], 1)
})
reading_scores_by_school

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [36]:
# print out the highest and lowest per-student budgets to decide on bins
print(f'Highest per-student budget: ${school_budget_norm_df.max():.2f}')
print(f'Lowest per-student budget: ${school_budget_norm_df.min():.2f}')

Highest per-student budget: $655.00
Lowest per-student budget: $578.00


In [49]:
# create bins - n bins with n + 1 bounds => 4 bins with 5 bounds
# I'll try $20 bins first since (655 - 578) / 4 = 19.25
spending_bins_20 = [0.0, 599.9, 619.9, 639.9, 9999.0]
spending_bins_20_labels = ['< $600', '$600-$619', '$620-']
# and matching the example bins to check my work:
spending_bins_example = [0.0, 584.9, 629.9, 644.9, 680.0]
spending_bins_example_labels = ['< $585', '$585-630', '$630-645', '$645-680']

# convert school_summary_df['Per_Student_Budget'] back to floats and add bin column
school_summary_df[school_budget_norm_df.name] = school_budget_norm_df
school_summary_df['Spending Range (Per Student)'] = pd.cut(school_summary_df['Per Student Budget'],
                               spending_bins_example,
                               labels= spending_bins_example_labels)

# set up the summary df's components:
scores_by_spending = school_summary_df.groupby('Spending Range (Per Student)').mean()[['Average Math Score',
                                                                                       'Average Reading Score',
                                                                                       '% Passing Math',
                                                                                       '% Passing Reading',
                                                                                       '% Overall Passing']]
# set up the summary df
scores_by_spending_formatted = pd.DataFrame({
    'Average Math Score': round(scores_by_spending['Average Math Score'], 1),
    'Average Reading Score': round(scores_by_spending['Average Reading Score'], 1),
    '% Passing Math': round(scores_by_spending['% Passing Math'], 1),
    '% Passing Reading': round(scores_by_spending['% Passing Reading'], 1),
    '% Overall Passing': round(scores_by_spending['% Overall Passing'], 1)
})
scores_by_spending_formatted

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Range (Per Student),,,,,
< $585,83.5,83.9,93.5,96.6,90.4
$585-630,81.9,83.2,87.1,92.7,81.4
$630-645,78.5,81.6,73.5,84.4,62.9
$645-680,77.0,81.0,66.2,81.1,53.5


## Scores by School Size

* Perform the same operations as above, based on school size.

In [55]:
# print out the highest and lowest per-student budgets to decide on bins
max_students = school_summary_df['Total Students'].max()
min_students = school_summary_df['Total Students'].min()
print(f'Largest School Population: {max_students}')
print(f'Smallest School Population: {min_students}')

Largest School Population: 4976
Smallest School Population: 427


In [56]:
# create bins - n bins with n + 1 bounds => 4 bins with 5 bounds
# I'll try $20 bins first since (655 - 578) / 4 = 19.25
size_bins = [0, 999, 1999, 9999]
size_bins_labels = ['Small (< 1000)', 'Medium (1000-1999)', 'Large (2000+)']

# add bin column to school_summary_df
school_summary_df['School Size'] = pd.cut(school_summary_df['Total Students'],
                               size_bins,
                               labels=size_bins_labels)

# set up the summary df's components:
scores_by_size = school_summary_df.groupby('School Size').mean()[['Average Math Score',
                                                                  'Average Reading Score',
                                                                  '% Passing Math',
                                                                  '% Passing Reading',
                                                                  '% Overall Passing']]
# set up the summary df
scores_by_size_formatted = pd.DataFrame({
    'Average Math Score': round(scores_by_size['Average Math Score'], 1),
    'Average Reading Score': round(scores_by_size['Average Reading Score'], 1),
    '% Passing Math': round(scores_by_size['% Passing Math'], 1),
    '% Passing Reading': round(scores_by_size['% Passing Reading'], 1),
    '% Overall Passing': round(scores_by_size['% Overall Passing'], 1)
})
scores_by_size_formatted

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (< 1000),83.8,83.9,93.6,96.1,89.9
Medium (1000-1999),83.4,83.9,93.6,96.8,90.6
Large (2000+),77.7,81.3,70.0,82.8,58.3


## Scores by School Type

* Perform the same operations as above, based on school type

In [59]:
scores_by_type = school_summary_df.groupby('School Type').mean()[['Average Math Score',
                                                                  'Average Reading Score',
                                                                  '% Passing Math',
                                                                  '% Passing Reading',
                                                                  '% Overall Passing']]
scores_by_type_formatted = pd.DataFrame({
    'Average Math Score': round(scores_by_type['Average Math Score'], 1),
    'Average Reading Score': round(scores_by_type['Average Reading Score'], 1),
    '% Passing Math': round(scores_by_type['% Passing Math'], 1),
    '% Passing Reading': round(scores_by_type['% Passing Reading'], 1),
    '% Overall Passing': round(scores_by_type['% Overall Passing'], 1)
})
scores_by_type_formatted

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.5,83.9,93.6,96.6,90.4
District,77.0,81.0,66.5,80.8,53.7
